In [1]:
## Dataset (Tips Dataset)
import seaborn as sns
df = sns.load_dataset('tips')

In [2]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [4]:
df['sex'].value_counts()
df['smoker'].value_counts()
# sex and smoker can be parsed through label encoding where each binary decision can be replaced by 0 or 1
df['day'].value_counts()
# day can be parsed through one hot encoding as here we have more values than 2
df['time'].value_counts()

# This encoding label and one hot encoding comes under FEATURE ENCODING

,count
time,
Dinner,176
Lunch,68


In [5]:
# independnent and dependent features
X = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y = df['total_bill']

In [6]:
##train test split
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.25 , random_state = 10)

In [7]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,Male,Yes,Sat,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
68,2.01,Male,No,Sat,Dinner,2
184,3.00,Male,Yes,Sun,Dinner,2


In [8]:
# Feature Encoding (Label Encoding & OneHot Encoding)
from sklearn.preprocessing import LabelEncoder

le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

In [9]:
# LABEL ENCODING
import warnings
warnings.filterwarnings('ignore')

# for binary features
X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])

X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [10]:
X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])

X_test.head()

,tip,sex,smoker,day,time,size
162,2.00,0,0,Sun,0,3
60,3.21,1,1,Sat,0,2
61,2.00,1,1,Sat,0,2
63,3.76,1,1,Sat,0,4
69,2.09,1,1,Sat,0,2


In [11]:
# One Hot encoding - Column Transformer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# from sksurv.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('onehot' , OneHotEncoder(drop = 'first') , [3])] , remainder = 'passthrough')

In [12]:
# import sys
# import numpy
# numpy.set_printoptions(threshold = sys.maxsize)
# X_train = ct.fit_transform(X_train)
# # ct.fit_transform(X_train)


import sys
import numpy
numpy.set_printoptions(threshold = sys.maxsize)
X_train = ct.fit_transform(X_train)

# Convert the sparse matrix to a dense array
# X_train = X_train.toarray()

# Now you can fit_transform
# X_train = ct.fit_transform(X_train)

In [13]:
X_test = ct.transform(X_test)

In [14]:
## SVR--Support Vector Regression
from sklearn.svm import SVR
svr=SVR()

In [15]:
svr.fit(X_train , y_train)

SVR()

In [16]:
y_pred = svr.predict(X_test)

In [17]:
from sklearn.metrics import r2_score , mean_absolute_error

print(r2_score(y_test , y_pred))
print(mean_absolute_error(y_test , y_pred))

0.46028114561159283
4.1486423210190235


# ***Hyperparameter Tuning using GridSearch CV***

In [18]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']}

In [19]:
grid = GridSearchCV(SVR() ,  param_grid ,  refit = True , verbose = 3)

# fitting the model for grid search
grid.fit(X_train , y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.025 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.021 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.010 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.124 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.050 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.053 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [20]:
grid.best_params_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

In [21]:
grid_prediction = grid.predict(X_test)

In [22]:
from sklearn.metrics import r2_score , mean_absolute_error
print(r2_score(y_test , grid_prediction))
print(mean_absolute_error(y_test , grid_prediction))

0.5081618245078687
3.8685147526100234
